# Cellpose in Python

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/05_segmentation/deep_learning/cellpose_notebook.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/05_segmentation/deep_learning/cellpose_notebook.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "tifffile",
#     "cellpose"
# ]
# ///

# Overview

In this section...

If you do not have an Apple Silicon Mac or a GPU we suggest to run this notebook in Google Colab. 

...

The images we will use for this section can be downloaded from the <a href="../../../_static/data/05_segmentation_cellpose.zip" download> <i class="fas fa-download"></i> Cellpose Dataset</a>.

## Import Libraries

In [43]:
import tifffile
from cellpose import core, io



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	darwin 
python version: 	3.13.0 
torch version:  	2.7.1! The neural network component of
CPSAM is much larger than in previous versions and CPU execution is slow. 
We encourage users to use GPU/MPS if available. 




## Setup

In [44]:
io.logger_setup()  # run this to get printing of progress

print("GPU available:", core.use_gpu())

2025-07-10 08:10:31,529 [INFO] WRITING LOG OUTPUT TO /Users/fdrgsp/.cellpose/run.log
2025-07-10 08:10:31,530 [INFO] 
cellpose version: 	4.0.6 
platform:       	darwin 
python version: 	3.13.0 
torch version:  	2.7.1
2025-07-10 08:10:31,547 [INFO] ** TORCH MPS version installed and working. **
GPU available: True


## Run Cellpose on A Sample Image

...

### Load the Image

...

In [41]:
image_path = "../../../_static/images/cellpose/cell_cellpose.tif"
image = tifffile.imread(image_path)

### Init the Model

...

### Run Cellpose

...

### Display the Results

## Run Cellpose on a Folder of Images

...

TO REMOVE: <a href="https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/test_Cellpose-SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>